In [ ]:
%pip install -U pip setuptools wheel
%pip install -U spacy
!python -m spacy download en_core_web_sm

In [2]:
import rich, json
import pandas as pd
from tqdm import tqdm
import spacy
from spacy import displacy, tokens
from spacy.tokens import DocBin
import os

In [ ]:
with open(f'{os.getcwd()}/labels_en.json','r') as f:
    data = json.load(f)

In [4]:
nlp = spacy.load("en_core_web_sm") 
doc_bin = DocBin()

In [ ]:
i = 0
for d in tqdm(data):
    text = data[i]["content"]
    labels = data[i]["labels"]
    doc = nlp.make_doc(text)

    spans = []
    for l in labels:
        #TODO

    doc.set_ents(spans)
    doc_bin.add(doc)
    i += 1

doc_bin.to_disk("train.spacy")